In [2]:
import numpy as np
import scanpy as sc
import matplotlib.pyplot as plt
import scipy.sparse
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_sample_weight

c:\Users\bence\Projects\BIO446\McKinnon-Rosati-Laboratory\env\Lib\site-packages\anndata\utils.py:429: FutureWarning: Importing read_csv from `anndata` is deprecated. Import anndata.io.read_csv instead.
  warnings.warn(msg, FutureWarning)
c:\Users\bence\Projects\BIO446\McKinnon-Rosati-Laboratory\env\Lib\site-packages\anndata\utils.py:429: FutureWarning: Importing read_excel from `anndata` is deprecated. Import anndata.io.read_excel instead.
  warnings.warn(msg, FutureWarning)
c:\Users\bence\Projects\BIO446\McKinnon-Rosati-Laboratory\env\Lib\site-packages\anndata\utils.py:429: FutureWarning: Importing read_hdf from `anndata` is deprecated. Import anndata.io.read_hdf instead.
  warnings.warn(msg, FutureWarning)
c:\Users\bence\Projects\BIO446\McKinnon-Rosati-Laboratory\env\Lib\site-packages\anndata\utils.py:429: FutureWarning: Importing read_loom from `anndata` is deprecated. Import anndata.io.read_loom instead.
  warnings.warn(msg, FutureWarning)
c:\Users\bence\Projects\BIO446\McKinnon-Ro

In [2]:
data = "C:\\Users\\bence\\Projects\\BIO446\\McKinnon-Rosati-Laboratory\\Project 1\\Data\\granulomas_final.h5ad"

In [3]:
adata = sc.read_h5ad(data)
adata.uns['log1p']["base"] = None # bug fix

print(adata.obs['sample'].value_counts())
print()
print('X matrix is sparse:', scipy.sparse.issparse(adata.X))
print('X size =', adata.X.shape)

sample
granuloma2    9392
granuloma1    9302
granuloma3    8909
Name: count, dtype: int64

X matrix is sparse: False
X size = (27603, 23693)


In [23]:
adata

AnnData object with n_obs × n_vars = 27603 × 23693
    obs: 'scDblFinder_score', 'scDblFinder_class', 'doublet', 'n_genes_by_counts', 'total_counts', 'type', 'sample', 'batch_type', 'batch', 'log1p_n_genes_by_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes', 'total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'pct_counts_ribo', 'total_counts_hb', 'pct_counts_hb', 'n_genes', 'n_counts', 'size_factors', 'leiden_0.2', 'leiden_0.3', 'leiden_0.4', 'leiden_0.6', 'leiden_0.8', 'leiden_1.0', 'leiden_1.4', 'leiden_1.7', 'leiden_2.0', 'leiden_2.5', 'leiden_3.0', 'leiden_3.5', 'leiden_4.0', 'leiden_4.5', 'hclust', 'kmeans', 'my_clust_1', 'immune', 'mesothelial', 'stromal', 'endothelial', 'epithelial', 'muscle', 'Immune', 'Myeloid', 'Lymphoid', 'Mesenchyme', 'Mesothelial', 'Stromal', 'SMC', 'Endothelial', 'Blood vessels', 'LEC', 'Epithelial', 'Ciliated', 'Secretory', 'cell_type_edit', 'cell

In [24]:
print(adata.X)
print(adata.X.shape)
print(type(adata.X))
print(adata.X.dtype)

[[0.         0.         0.         ... 0.         0.         0.65802306]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.9530956 ]
 ...
 [0.         0.         0.         ... 0.         0.         0.40804082]
 [0.         0.         0.         ... 0.         0.39723676 1.0820873 ]
 [0.         0.         0.         ... 0.         0.         0.        ]]
(27603, 23693)
<class 'numpy.ndarray'>
float32


In [ ]:
print(adata.obsm["Scanorama"])
print(adata.obsm["Scanorama"].shape)
print(type(adata.obsm["Scanorama"]))
print(adata.obsm["Scanorama"].dtype)

In [ ]:
print(adata.obsm["X_pca"])
print(adata.obsm["X_pca"].shape)
print(type(adata.obsm["X_pca"]))
print(adata.obsm["X_pca"].dtype)

In [ ]:
print(adata.obsm["X_umap"])
print(adata.obsm["X_umap"].shape)
print(type(adata.obsm["X_umap"]))
print(adata.obsm["X_umap"].dtype)

In [4]:
cluster_type = 'my_clust_1'

In [5]:
adata.obs[cluster_type]

AAACCTGAGCGTAGTG_sc69_1     6
AAACCTGAGCTGATAA_sc69_1     3
AAACCTGCAAATCCGT_sc69_1     4
AAACCTGCAAGGGTCA_sc69_1    13
AAACCTGCACATCCGG_sc69_1     4
                           ..
TTTGTCAGTTCAACCA_sc72_1     4
TTTGTCAGTTCAGTAC_sc72_1     8
TTTGTCATCACAAACC_sc72_1     4
TTTGTCATCAGCACAT_sc72_1     4
TTTGTCATCTCGAGTA_sc72_1    22
Name: my_clust_1, Length: 27603, dtype: category
Categories (30, object): ['0', '10', '11', '11b', ..., '7c', '8', '9', '9b']

In [6]:
annotation_dict= {
    '9': 'CAP1',
    '24': 'CAP2',
    '9b': 'VEC',
    '27': 'LEC',
        
    '17': 'Ciliated',
    '15': 'Secretory',
    '22': 'AT1',
    '6': 'AT2',
    '12': 'AT2-t1',
    '19': 'AT2-t2',
        
    '14': 'AF',
    '25': 'Pericyte',
        
    '20': 'Mesothelial',
        
    '3': 'B1',
    '3b': 'B2',
        
    '0': 'Th1',
    '8': 'Tnaive',
    '11': 'Tex',
    '77': 'Treg',
        
    '11b': 'NK',
        
    '4a': 'AM',
    '4': 'M-t1',
    '10': 'M-lc',
    '7': 'M-t2',
    '7b': 'M-C1q',
    '7c': 'iMon',
        
    '23': 'pDC',
    '13': 'DC',
    '5b': 'N1',
    '5': 'N2',
}

In [7]:
adata.obs['cell_type_edit'] = [annotation_dict[clust] for clust in adata.obs[cluster_type]]

dict_list = list(annotation_dict.keys())
adata_list = list(adata.obs[cluster_type].unique())
print('Keys in dictionary not in adata:', [item for item in dict_list if item not in adata_list] )
print('Keys in adata not in dictionary:', [item for item in adata_list if item not in dict_list] )

Keys in dictionary not in adata: []
Keys in adata not in dictionary: []


In [8]:
adata.obs['cell_type_edit']

AAACCTGAGCGTAGTG_sc69_1       AT2
AAACCTGAGCTGATAA_sc69_1        B1
AAACCTGCAAATCCGT_sc69_1      M-t1
AAACCTGCAAGGGTCA_sc69_1        DC
AAACCTGCACATCCGG_sc69_1      M-t1
                            ...  
TTTGTCAGTTCAACCA_sc72_1      M-t1
TTTGTCAGTTCAGTAC_sc72_1    Tnaive
TTTGTCATCACAAACC_sc72_1      M-t1
TTTGTCATCAGCACAT_sc72_1      M-t1
TTTGTCATCTCGAGTA_sc72_1       AT1
Name: cell_type_edit, Length: 27603, dtype: object

In [9]:
unique_celltype_sub = adata.obs['cell_type_edit'].unique()
print(unique_celltype_sub)

num_unique_celltype_sub = adata.obs['cell_type_edit'].nunique()
print(f"Number of unique sub cell types: {num_unique_celltype_sub}")

['AT2' 'B1' 'M-t1' 'DC' 'Th1' 'M-t2' 'Secretory' 'AM' 'N1' 'M-C1q'
 'AT2-t2' 'AF' 'VEC' 'CAP1' 'N2' 'AT2-t1' 'Pericyte' 'pDC' 'Ciliated' 'NK'
 'AT1' 'Tnaive' 'Treg' 'M-lc' 'Mesothelial' 'Tex' 'CAP2' 'LEC' 'iMon' 'B2']
Number of unique sub cell types: 30


In [10]:
replacement_dict = {
    'AT2': 0,
    'B1': 1,
    'M-t1': 2,
    'DC': 3,
    'Th1': 4,
    'M-t2': 5,
    'Secretory': 6,
    'AM': 7,
    'N1': 8,
    'M-C1q': 9,
    'AT2-t2': 10,
    'AF': 11,
    'VEC': 12,
    'CAP1': 13,
    'N2': 14,
    'AT2-t1': 15,
    'Pericyte': 16,
    'pDC': 17,
    'Ciliated': 18,
    'NK': 19,
    'AT1': 20,
    'Tnaive': 21,
    'Treg': 22,
    'M-lc': 23,
    'Mesothelial': 24,
    'Tex': 25,
    'CAP2': 26,
    'LEC': 27,
    'iMon': 28,
    'B2': 29
}

In [11]:
adata.obs['celltype'] = adata.obs['cell_type_edit'].replace(replacement_dict)

C:\Users\bence\AppData\Local\Temp\ipykernel_60976\975682991.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  adata.obs['celltype'] = adata.obs['cell_type_edit'].replace(replacement_dict)


In [12]:
adata.obs['celltype']

AAACCTGAGCGTAGTG_sc69_1     0
AAACCTGAGCTGATAA_sc69_1     1
AAACCTGCAAATCCGT_sc69_1     2
AAACCTGCAAGGGTCA_sc69_1     3
AAACCTGCACATCCGG_sc69_1     2
                           ..
TTTGTCAGTTCAACCA_sc72_1     2
TTTGTCAGTTCAGTAC_sc72_1    21
TTTGTCATCACAAACC_sc72_1     2
TTTGTCATCAGCACAT_sc72_1     2
TTTGTCATCTCGAGTA_sc72_1    20
Name: celltype, Length: 27603, dtype: int64

In [13]:
print("Unique values in 'celltype' column after replacement:", adata.obs['celltype'].unique())

Unique values in 'celltype' column after replacement: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29]


In [14]:
cell_type_counts = adata.obs['cell_type_edit'].value_counts()

print("Number of cells in each cell type:")
print(cell_type_counts)

Number of cells in each cell type:
cell_type_edit
M-t1           5692
Th1            3420
B1             2068
AT2            1821
N2             1819
Tnaive         1226
M-lc           1162
M-t2           1094
AF             1091
AM              996
AT2-t1          871
DC              774
CAP1            740
Secretory       696
VEC             439
Ciliated        401
AT2-t2          379
Mesothelial     362
M-C1q           357
AT1             340
Tex             296
pDC             286
NK              251
CAP2            213
Treg            203
N1              160
B2              140
Pericyte        138
iMon            111
LEC              57
Name: count, dtype: int64


In [ ]:
# cell distribution with cell types
y = adata.obs['cell_type_edit']
plt.figure(figsize=(35, 18))
plt.title('Cell Distribution with Cell Types')
y.value_counts().plot(kind='bar', rot=0)
plt.show()

In [ ]:
# cell distribution with int mappings
y = adata.obs['celltype']
plt.figure(figsize=(14, 8))
plt.title('Cell Distribution with Integer Mappings')
y.value_counts().plot(kind='bar', rot=0)
plt.show()

In [ ]:
# trying to confirm that adata.X indices correspond to adata.obs indices
# the idea is that the gene expression levels should correlate to the total counts of each cell
# not accurate with adata.X because those sums have been log transformed

row_sums_X = np.array(adata.layers["counts"].sum(axis=1)).flatten() # adata.layers["counts"] should be the raw counts matrix (maybe check granulomas_final_raw.h5ad's adata.X for equality)

obs_sums = adata.obs["total_counts"].values

corr = np.corrcoef(row_sums_X, obs_sums)[0,1]
print("Correlation:", corr)

Correlation: 0.9996135210465931


In [ ]:
fourth_dict = {
    'Blood vessels': ['CAP1','CAP2','VEC','AEC'],
    'Lymphatic EC': ['LEC'],
    'Airway epithelium': ['Ciliated','Secretory'],
    'Alveolar epithelium': ['AT1','AT2','AT2-t1','AT2-t2'],
    'Fibroblast': ['AF','Pericyte'],
    'Smooth muscle': ['SMC'],
    'Mesothelial': ['Mesothelial'],
    'B lineage': ['B1','B2'],
    'T lineage': ['Th1','Tnaive','Treg','Tex'],
    'NK': ['NK'],
    'Macrophage': ['AM','M-t1','M-t2','M-C1q','M-lc'],        
    'mononuclear': ['iMon','DC','pDC'],
    'Neutrophil': ['N1','N2']
}

group_lookup = {}
for broad_cat, fine_types in fourth_dict.items():
    for ft in fine_types:
        group_lookup[ft] = broad_cat

In [ ]:
group_lookup

In [ ]:
adata.obs['broad_celltype'] = adata.obs['cell_type_edit'].map(group_lookup)

In [ ]:
adata.obs['broad_celltype']

In [ ]:
print("Broad celltype value counts:")
print(adata.obs['broad_celltype'].value_counts())
print(adata.obs['broad_celltype'].unique())

In [ ]:
broad_types = adata.obs['broad_celltype'].unique()
cell_type_subsets = {}

for cat in broad_types:
    
    adata_sub = adata[adata.obs['broad_celltype'] == cat].copy()
    cell_type_subsets[cat] = adata_sub
    
    print(f"\nCreated subset for broad cell type '{cat}' "f"with {adata_sub.n_obs} cells.")

In [ ]:
adata_alveolar_epithelium = cell_type_subsets['Alveolar epithelium']
adata_b_lineage = cell_type_subsets['B lineage']
adata_macrophage = cell_type_subsets['Macrophage']
adata_mononuclear = cell_type_subsets['mononuclear']
adata_t_lineage = cell_type_subsets['T lineage']
adata_airway_epithelium = cell_type_subsets['Airway epithelium']
adata_neutrophil = cell_type_subsets['Neutrophil']
adata_fibroblast = cell_type_subsets['Fibroblast']
adata_blood_vessels = cell_type_subsets['Blood vessels']
adata_nk = cell_type_subsets['NK']
adata_mesothelial = cell_type_subsets['Mesothelial']
adata_lymphatic_ec = cell_type_subsets['Lymphatic EC']

In [ ]:
adata_alveolar_epithelium.write("C:\\Users\\bence\\Projects\\BIO446\\McKinnon-Rosati-Laboratory\\Project 1\\Data\\adata_alveolar_epithelium.h5ad") 
adata_b_lineage.write("C:\\Users\\bence\\Projects\\BIO446\\McKinnon-Rosati-Laboratory\\Project 1\\Data\\adata_b_lineage.h5ad")
adata_macrophage.write("C:\\Users\\bence\\Projects\\BIO446\\McKinnon-Rosati-Laboratory\\Project 1\\Data\\adata_macrophage.h5ad")
adata_mononuclear.write("C:\\Users\\bence\\Projects\\BIO446\\McKinnon-Rosati-Laboratory\\Project 1\\Data\\adata_mononuclear.h5ad")
adata_t_lineage.write("C:\\Users\\bence\\Projects\\BIO446\\McKinnon-Rosati-Laboratory\\Project 1\\Data\\adata_t_lineage.h5ad")
adata_airway_epithelium.write("C:\\Users\\bence\\Projects\\BIO446\\McKinnon-Rosati-Laboratory\\Project 1\\Data\\adata_airway_epithelium.h5ad")
adata_neutrophil.write("C:\\Users\\bence\\Projects\\BIO446\\McKinnon-Rosati-Laboratory\\Project 1\\Data\\adata_neutrophil.h5ad")
adata_fibroblast.write("C:\\Users\\bence\\Projects\\BIO446\\McKinnon-Rosati-Laboratory\\Project 1\\Data\\adata_fibroblast.h5ad")
adata_blood_vessels.write("C:\\Users\\bence\\Projects\\BIO446\\McKinnon-Rosati-Laboratory\\Project 1\\Data\\adata_blood_vessels.h5ad")
adata_nk.write("C:\\Users\\bence\\Projects\\BIO446\\McKinnon-Rosati-Laboratory\\Project 1\\Data\\adata_nk.h5ad")
adata_mesothelial.write("C:\\Users\\bence\\Projects\\BIO446\\McKinnon-Rosati-Laboratory\\Project 1\\Data\\adata_mesothelial.h5ad")
adata_lymphatic_ec.write("C:\\Users\\bence\\Projects\\BIO446\\McKinnon-Rosati-Laboratory\\Project 1\\Data\\adata_lymphatic_ec.h5ad")

In [ ]:
# highly variable genes

adata_hv = adata[:, adata.var['highly_variable'] ].copy()
print("# highly variable genes:", adata.var['highly_variable'].sum())
print("shape:", adata_hv.shape)

In [16]:
# scGNN imputed matrix
scGNN_matrix = sc.read_csv("../Data/granuloma_recon.csv")

In [17]:
scGNN_matrix

AnnData object with n_obs × n_vars = 27603 × 23693

In [18]:
scGNN_matrix.obs["celltype"] = adata.obs["celltype"]

In [20]:
scGNN_matrix.obs["celltype"]

AAACCTGAGCGTAGTG_sc69_1     0
AAACCTGAGCTGATAA_sc69_1     1
AAACCTGCAAATCCGT_sc69_1     2
AAACCTGCAAGGGTCA_sc69_1     3
AAACCTGCACATCCGG_sc69_1     2
                           ..
TTTGTCAGTTCAACCA_sc72_1     2
TTTGTCAGTTCAGTAC_sc72_1    21
TTTGTCATCACAAACC_sc72_1     2
TTTGTCATCAGCACAT_sc72_1     2
TTTGTCATCTCGAGTA_sc72_1    20
Name: celltype, Length: 27603, dtype: int64

In [21]:
# not accurate because scGNN_matrix.X has transformed the sums in some way

row_sums_X = np.array(scGNN_matrix.X.sum(axis=1)).flatten()

obs_sums = adata.obs["total_counts"].values

corr = np.corrcoef(row_sums_X, obs_sums)[0,1]
print("Correlation:", corr)

Correlation: 0.6252261824452869


In [22]:
scGNN_matrix.write("../Data/scGNN_matrix.h5ad") 

In [17]:
scGNN_embedding = sc.read_csv("../Data/granuloma_embedding.csv")

In [18]:
scGNN_embedding

AnnData object with n_obs × n_vars = 23693 × 128

In [29]:
scGNN_embedding.obs

""
Xkr4
Gm1992
Gm19938
Gm37381
Rp1
...
CR974586.2
CR974586.4
4933409K07Rik
CAAA01147332.1


In [30]:
scGNN_embedding.var

""
embedding0
embedding1
embedding2
embedding3
embedding4
...
embedding123
embedding124
embedding125
embedding126


In [ ]:
X = adata.X
y = adata.obs['celltype'].values

# random states (completely arbitrary):
# adata.X -> (19, 30)
# adata.obsm["Scanorama"] -> (8, 3)
# adata.obsm["X_pca"] -> (3, 13)
# adata.obsm["X_umap"] -> (34, 90)
# adata_hv.X -> (6, 7)

train_features, test_features, train_labels, test_labels = train_test_split(X, y, test_size = 0.2, random_state=19)
train_features, val_features, train_labels, val_labels = train_test_split(train_features, train_labels, test_size = 0.25, random_state=30)

train_features = np.array(train_features)
test_features = np.array(test_features)
val_features = np.array(val_features)

train_labels = np.array(train_labels)
test_labels = np.array(test_labels)
val_labels = np.array(val_labels)

print('Training features shape:', train_features.shape)
print('Validation features shape:', val_features.shape)
print('Test features shape:', test_features.shape)

print('Training labels shape:', train_labels.shape)
print('Validation labels shape:', val_labels.shape)
print('Test labels shape:', test_labels.shape)

sample_weights = compute_sample_weight(
    class_weight='balanced',
    y=train_labels
)

print('Sample weights shape:', sample_weights.shape)

np.save('../Arrays/train_features.npy', train_features)
np.save('../Arrays/test_features.npy', test_features)
np.save('../Arrays/val_features.npy', val_features)
np.save('../Arrays/train_labels.npy', train_labels)
np.save('../Arrays/test_labels.npy', test_labels)
np.save('../Arrays/val_labels.npy', val_labels)
np.save('../Arrays/sample_weights.npy', sample_weights)

In [ ]:
# 5 GB
adata.write("C:\\Users\\bence\\Projects\\BIO446\\McKinnon-Rosati-Laboratory\\Project 1\\Data\\adata.h5ad")

In [ ]:
# for umap display
indices = train_test_split(adata.obs.index, test_size=0.2, random_state=19)[1]
indices = np.array(indices)
np.save('../Arrays/indices.npy', indices)

In [26]:
adata.X.max()

10.413485